In [33]:
import pandas as pd 

In [34]:
# read in the oecd data
df = pd.read_csv('data/gdpph_oecd.csv')

df = df[['Country', 'Measure','TIME_PERIOD', 'OBS_VALUE']]

df.columns = ['country', 'measure', 'year', 'value']

df = df[df['measure'] == '2015=100']

df = df.drop(columns=['measure'])

# sort by country and year
df = df.sort_values(by=['country', 'year'])

df

,country,year,value
1435,Australia,1990,65.728880
1371,Australia,1991,67.239157
472,Australia,1992,69.306729
502,Australia,1993,69.456332
48,Australia,1994,69.609837
...,...,...,...
2215,United States,2018,102.342200
1305,United States,2019,103.581713
1239,United States,2020,107.111176
236,United States,2021,108.386615


In [35]:
# convert year to datetime (first day of the year)
df['year'] = pd.to_datetime(df['year'], format='%Y')

df

,country,year,value
1435,Australia,1990-01-01,65.728880
1371,Australia,1991-01-01,67.239157
472,Australia,1992-01-01,69.306729
502,Australia,1993-01-01,69.456332
48,Australia,1994-01-01,69.609837
...,...,...,...
2215,United States,2018-01-01,102.342200
1305,United States,2019-01-01,103.581713
1239,United States,2020-01-01,107.111176
236,United States,2021-01-01,108.386615


In [36]:
df.to_csv('data/gdpph_clean.csv', index=False)